In [1]:
import os

import pandas as pd
import numpy as np

import scipy.stats as stats
from scipy.stats import ttest_ind

from datetime import datetime, timedelta

from tqdm.notebook import tqdm

In [10]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-13/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# Задание 1

В лекции мы выяснили, что в эксперименте «Refactoring backend» в экспериментальной группе среднее время загрузки увеличилось, а 99% квантиль уменьшился. Проверьте значимость отличий других квантилей.

Данные эксперимента «Refactoring backend»: 2022-04-13/2022-04-13T12_df_web_logs.csv и 2022-04-13/experiment_users.csv. Эксперимент проводился с 2022-04-05 по 2022-04-12. Измерения времени обработки запросов считаем независимыми. При проверке используйте нормальный доверительный интервал.

In [11]:
web_logs = read_database('2022-04-13T12_df_web_logs.csv')
web_logs['date'] = pd.to_datetime(web_logs['date'])
web_logs.head(3)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7


In [12]:
experiment_users = read_database('experiment_users.csv')
experiment_users

,user_id,pilot
0,c36b2e,0
1,20336e,0
2,034652,0
3,e98e3b,0
4,3f1105,0
...,...,...
5451,7b4bba,1
5452,923ad6,1
5453,6e59e9,1
5454,bb8df6,1


In [16]:
web_logs_exp = web_logs[(web_logs['date'] >= datetime(2022, 4, 5)) & (web_logs['date'] <= datetime(2022, 4, 12))]
web_logs_exp = web_logs_exp.merge(experiment_users, how='inner', on='user_id')

In [17]:
web_logs_exp

,user_id,page,date,load_time,pilot
0,e65269,m,2022-04-05 00:01:25,60.0,1
1,e65269,m,2022-04-08 08:32:38,71.7,1
2,e65269,m,2022-04-08 08:45:27,66.1,1
3,e65269,b,2022-04-08 08:47:59,73.6,1
4,e65269,m,2022-04-08 08:50:37,68.7,1
...,...,...,...,...,...
38931,d532c9,m,2022-04-11 22:33:09,84.1,0
38932,d532c9,m,2022-04-11 22:47:01,73.3,0
38933,795c7b,m,2022-04-11 22:40:42,76.5,1
38934,795c7b,m,2022-04-11 22:45:43,70.9,1


In [24]:
load_time_a =  web_logs_exp.loc[web_logs_exp['pilot'] == 0, 'load_time'].values
load_time_b =  web_logs_exp.loc[web_logs_exp['pilot'] == 1, 'load_time'].values

In [25]:
def get_point_estimate(a, b, quantile):
    return np.quantile(b, quantile) - np.quantile(a, quantile)

In [33]:
alpha = 0.05

quantiles = [0.7, 0.74, 0.78, 0.82, 0.86, 0.9, 0.95, 0.99, 0.999, 0.9999]

results = []
for quantile in quantiles:
    point_estimate = get_point_estimate(load_time_a, load_time_b, quantile)

    bs_metric = []
    for i in range(1000):
        bs_load_time_a = np.random.choice(load_time_a, size=len(load_time_a), replace=True)
        bs_load_time_b = np.random.choice(load_time_b, size=len(load_time_b), replace=True)
        bs_diff = get_point_estimate(bs_load_time_a, bs_load_time_b, quantile)
        bs_metric.append(bs_diff)

    std = np.std(bs_metric)

    left, right = point_estimate - stats.norm.ppf(1 - alpha / 2) * std, point_estimate + stats.norm.ppf(1 - alpha / 2) * std
    res = 1 if (left > 0) or (right < 0) else 0
    results.append(res)
print(results)

[1, 1, 0, 1, 1, 1, 1, 1, 0, 0]


In [35]:
np.quantile([0,1,2,3,4,5], [0.1, 0.9])

array([0.5, 4.5])